In [1]:
from pymongo import MongoClient
from urllib.request import urlopen
from bs4 import BeautifulSoup
import lxml

DB_NAME = "advice_articles"
COL_NAME = "dear_prudence"

client = MongoClient()
db = client[DB_NAME]
col = db[COL_NAME] 

In [2]:
from scrapy.selector import Selector
from scrapy.http import HtmlResponse

loc = col.find_one()['loc']
html = urlopen(loc).read()
selector = Selector(text=html)

In [7]:
print(col.find_one())

{'_id': ObjectId('5995b6374ea378188f1ccfc1'), 'loc': 'http://www.slate.com/articles/life/dear_prudence/2016/03/dear_prudence_should_i_have_a_child_so_it_will_care_for_me_one_day.single.html', 'lastmod': datetime.datetime(2016, 3, 8, 20, 34)}


In [3]:
article = selector.xpath("//article[@id='story-0']")

In [10]:
def parse_article_page(article_selector):
    article = article_selector.xpath("//article[@id='story-0']")
    all_paragraphs = article.xpath("//div[contains(@class, 'parbase')]")
    article_delimiters = []
    temp_delimiter = dict()
    
    
    parsed_articles = {
        "title" : article_selector.xpath("//h1[@class='hed']/text()").extract_first(),
        "subtitle" : article_selector.xpath("//h2[@class='dek']/text()").extract_first(),
        "author" : article_selector.xpath("//a[@rel='author']/text()").extract_first(),
        "questions" : []
    }

    # Get indices for start/end of questions and end of answers     
    for index in range(len(all_paragraphs)):
        element = all_paragraphs[index]
        title = element.xpath(".//p/strong/text()").extract_first()
        string = element.xpath(".//p/text()").extract_first()

        if title and title.startswith("Dear Prudence,"):
            temp_delimiter["start_index"] = index
        elif string and string.startswith("—"):
            temp_delimiter['end_question'] = index
        else:
            strong = element.xpath(".//strong/text()").extract_first()
            soft = element.xpath(".//p/text()").extract_first()
            if "start_index" in temp_delimiter.keys() and string == '* * *':
                temp_delimiter['end_index'] = index
                article_delimiters.append(temp_delimiter)
                temp_delimiter = dict()

    # Extract the articles based on the indices
    for delim_dict in article_delimiters:
        parsed_article = dict()

        question = ""
        alias = None
        answer = ""

        q_block = all_paragraphs[delim_dict['start_index']:delim_dict['end_question']]
        alias = all_paragraphs[delim_dict['end_question']].xpath(".//p/text()").extract_first()[1:]
        a_block = all_paragraphs[delim_dict['end_question']+1:delim_dict['end_index']]

        for p in q_block:
            question += p.xpath(".//p").extract_first()

        for p in a_block:
            answer += p.xpath(".//p").extract_first()

        parsed_article['question'] = question
        parsed_article['alias'] = alias
        parsed_article['answer'] = answer

        parsed_articles["questions"].append(parsed_article)
        
    return parsed_articles

In [11]:
parse_article_page(article)

{'author': 'Mallory Ortberg',
 'questions': [{'alias': 'Don’t Want to Die Alone',
   'answer': '<p><strong>I have a lot of sympathy for anyone </strong>willing to admit they’re terrified of dying alone. But you should not have children. Raising a child for the sole purpose of creating a future nurse is no reason to start a family. Any child of yours would pretty quickly pick up on the fact that you find them exhausting, expensive, and a burden—things that are true of all children, but in your case would not be balanced by parental love. My guess is that child would not then leap at the chance to provide you with round-the-clock care in your declining years.</p><p>You must know on some level that having children does not guarantee anyone a peaceful death. Many people outlive their children. Many people with living, healthy children are still put into nursing homes for a variety of reasons. Having a child now is not insurance that you will be given the kind of death you want. Address you